# OpenAI Async Example

We are going to create a simple chatbot that creates stories based on a user provided image. The chatbot will use the gpt-4o-mini LLM to generate the story using a user prompt and its vision model to understand the image.

We will track the chatbot with AgentOps and see how it performs!

First let's install the required packages

In [ ]:
%pip install -U openai
%pip install -U agentops

Then import them

In [1]:
from openai import AsyncOpenAI
import agentops
import os
from dotenv import load_dotenv

Next, we'll grab our API keys. You can use dotenv like below or however else you like to load environment variables

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

Next we initialize the AgentOps client.

In [ ]:
agentops.init(AGENTOPS_API_KEY, default_tags=["openai-async-example"])

And we are all set! Note the seesion url above. We will use it to track the chatbot.

Let's create a simple chatbot that generates stories given an image and a user prompt.

In [4]:
client = AsyncOpenAI(api_key=OPENAI_API_KEY)

system_prompt = """
You are a master storyteller, with the ability to create vivid and engaging stories.
You have experience in writing for children and adults alike.
You are given a prompt and you need to generate a story based on the prompt.
"""

user_prompt = [
    {"type": "text", "text": "Write a mystery thriller story based on your understanding of the provided image."},
    {
        "type": "image_url",
        "image_url": {"url": "https://www.cosy.sbg.ac.at/~pmeerw/Watermarking/lena_color.gif"},
    },
]

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

In [5]:
async def main():
    response = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )

    print(response.choices[0].message.content)

In [ ]:
await main()

The response is a string that contains the story. We can track this with AgentOps by navigating to the trace url and viewing the run.

## Streaming Version
We will demonstrate the streaming version of the API.

In [7]:
async def main_stream():
    stream = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True,
    )

    async for chunk in stream:
        print(chunk.choices[0].delta.content or "", end="")

In [ ]:
await main_stream()

Note that the response is a generator that yields chunks of the story. We can track this with AgentOps by navigating to the trace url and viewing the run.
All done!